----------
##### import Pororo

In [1]:
import pororo
from pororo import Pororo

/data/ephemeral/home/shark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------
##### 필요 Import

In [42]:
#import 모음

import nltk
import torch
import random
from datasets import load_from_disk, Dataset, DatasetDict, concatenate_datasets


----------
##### 초기 설정
- nltk.download 진행 시 'punkt'가 아닌 'punkt_tab'을 입력  
▶ 'punkt' 입력할 경우 mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")에서 오류 발생

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('punkt_tab')
mt = Pororo(task="translation", lang="multi", model='transformer.large.multi.mtpg')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


----------
##### 데이터 불러오기

In [15]:
disk = "/data/ephemeral/home/level2-mrc-nlp-12/data"

train_data = load_from_disk(disk + "/train_dataset/")
train_dataset = train_data["train"]
print(len(train_data['train']))

3952


----------
##### 역번역

In [28]:
def back_translation(text):
    return mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")

In [29]:
text = "케빈은 집에 갔다. 그러나 더 놀고 싶어한다."
test = back_translation(text)
print(test)

케빈은 집에 갔다 하지만 나는 더 뛰고 싶다


----------
##### 증강

In [30]:
def augment_data(train_dataset, limit):
    augmented_examples = []
    for idx, example in enumerate(train_dataset):
        if idx >= limit:
            break
        original_question = example['question']
        
        #역번역
        back_translated_question = back_translation(original_question)
        
        # 새로운 예제 생성
        new_example = example.copy()
        new_example['question'] = back_translated_question
        
        augmented_examples.append(new_example)
    
    # 리스트를 딕셔너리로 변환하여 Dataset 생성
    aug_train_data_dict = {key: [ex[key] for ex in augmented_examples] for key in augmented_examples[0]}
    aug_train_data = Dataset.from_dict(aug_train_data_dict)
    
    return aug_train_data

In [34]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

aug_train_data = augment_data(train_dataset, augmentation_limit)

----------
##### 데이터 출력 (상위 10개)

In [39]:
for idx, (original, augmented) in enumerate(zip(train_dataset, aug_train_data)):
    if idx >= 10:
        break
    print(f"기존 질문  : {original['question']}")
    print(f"역번역 질문: {augmented['question']}\n")

기존 질문  : 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
역번역 질문: 어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?

기존 질문  : 현대적 인사조직관리의 시발점이 된 책은?
역번역 질문: 현대 인사 운영의 출발점이 어떤 책이 됐을까?

기존 질문  : 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
역번역 질문: 강희제가 누구를 위해 1717년에 썼나요?

기존 질문  : 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
역번역 질문: 보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.

기존 질문  : 명문이 적힌 유물을 구성하는 그릇의 총 개수는?
역번역 질문: 그 이름으로 작성된 유물의 전체 병은 몇 개입니까?

기존 질문  : 카드모스의 부하들이 간 곳에는 무엇이 있었는가?
역번역 질문: 카드모스 남자들이 어디로 갔지?

기존 질문  : 관우를 불태워 죽이려한 사람 누구인가?
역번역 질문: 관을 태우려던 사람은 누구입니까?

기존 질문  : 참호 속에 무기와 장비를 버리고 도주한 집단은?
역번역 질문: 재난 속에서 무기와 장비를 버린 사람들이 한 무리인가요?

기존 질문  : 제2차 세계 대전에 참전하여 사망한 자식은?
역번역 질문: 2차 세계대전에서 누가 죽었나요?

기존 질문  : 고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?
역번역 질문: 고려왕은 어떤 마을에서 아내의 가족과 아내를 피난처로 만들었을까?



##### 랜덤한 30개 출력

In [40]:
random_indices = random.sample(range(len(train_dataset)), 30)

# 원래 질문과 역번역된 질문 출력 (랜덤 30개)
for idx in random_indices:
    original = train_dataset[idx]
    augmented = aug_train_data[idx]
    print(f"기존 질문  : {original['question']}")
    print(f"역번역 질문: {augmented['question']}\n")

기존 질문  : 엘리노어 여사가 사망한 해는?
역번역 질문: 엘리노 부인은 몇 년 동안 죽었습니까?

기존 질문  : 원희룡이 사상적 전환을 맞게 된 계기가 된 것은?
역번역 질문: 원희룡이 이념적 전환에 직면할 수 있는 기회인가?

기존 질문  : 잘록한 허리가 두드러지게 옷을 입는 방법은?
역번역 질문: 날카로운 허리로 옷을 입을 방법이 어떻게 되나요?

기존 질문  : 기후 변화 대응 체계 해소 취지에 선구할 수 있는 방안으로 대두되는 농업 장소는?
역번역 질문: 기후변화 대응체계 해결을 위한 선제적 대책으로 농업현장이 어디로 떠오르고 있는가?

기존 질문  : 6월 28일에 북한은 어디를 점거하였는가?
역번역 질문: 북한은 6월 28일 어디서 인수했습니까?

기존 질문  : 이자성이 산해군을 공격할 때 동원한 군사의 수는?
역번역 질문: 이자성이 상하이 육군을 공격했을 때 동원한 병사들은 몇 명인가?

기존 질문  : 에우포르보스와 아폴론이 함께 공격했던 인물은?
역번역 질문: 토르보스와 아폴로에게 공격당한 사람은 누구였습니까?

기존 질문  : 꼬리가 있는 도롱뇽과 달리 개구리는 분추류에서 갈라져 나왔다고 제시한 사람은?
역번역 질문: 미세한 도론과 달리 개구리가 낙하산에서 분리됐다고 제안한 사람은 누구인가?

기존 질문  : 신탁통치 오보사건은 어떤 신문사와 관련되어있나요?
역번역 질문: 어느 신문이 신뢰의 잘못된 정보에 관여하고 있는가?

기존 질문  : 로블링이 왕복선에 의해 다친 부위는?
역번역 질문: 로밍이 셔틀에 부상을 입은 지역은 어디 있지?

기존 질문  : S.롱스타드 중위의 국적은?
역번역 질문: 롱스타드 중위의 국적은 무엇입니까?

기존 질문  : 라파엘의 10월14일 오전 12시경의 풍속은?
역번역 질문: 라파엘의 풍속은 10월 14일 오전 12시쯤 어떤가?

기존 질문  : 1801년 상산군부인 송마리아와 평산군부인 신마리아가 죽임을 당한 일과 관련있는 종교는?
역번역 질문: 상산군 여성 송마리아와 평산군 신마리아가 1801년 

----------
##### 새로운 disk에 저장

In [43]:
combined_data = DatasetDict({'train': concatenate_datasets([train_dataset, aug_train_data])})
combined_data.save_to_disk(disk + "/pororo_train_data/")

Saving the dataset (1/1 shards): 100%|██████████| 7904/7904 [00:00<00:00, 219420.33 examples/s]


In [46]:
combined_data['train'][7341]

TypeError: 'DatasetInfo' object is not callable